In [53]:
import pandas as pd
import sys,os,copy,pdb,importlib
sys.path.append('../classes')
sys.path.append('../analysis')
import numpy as np
import matplotlib.pyplot as plt

import tournament_new as tn
import create_database as cd
importlib.reload(tn)

import beck.beck_game
from importlib import reload
reload(beck.beck_game)
from beck.beck_game import BeckGame as Game
game = Game(4,9,4)

from arena import Arena
from mcts import MCTS
importlib.reload(tn)
game = Game(4,9,4)
all_p = pd.read_pickle(cd.DATABASE_LOC)

# res = tn.merge_res_to_base()
tournament_res = pd.read_pickle('/scratch/zz737/fiar/tournaments/ai_all_player_round_robin_base.pkl')

import argparse
import os
import shutil
import time
import random
import numpy as np
import math
import sys

import keras
import keras.backend as K
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras import metrics

sys.path.append('..')
from neural_net import NeuralNet
from utils import *

reload(beck.beck_nnet)
from beck.beck_nnet import OthelloNNet, NNetWrapper

import beck.beck_nnet_pvcorr as bnp
reload(bnp)

<module 'beck.beck_nnet_pvcorr' from '../classes/beck/beck_nnet_pvcorr.py'>

In [58]:
# load and organize data
from pickle import Pickler, Unpickler
# weight_dir = '/scratch/zz737/fiar/tournaments/tournament_6/checkpoints_mcts100_cpuct2_id_0'
weight_dir = '/scratch/zz737/fiar/tournaments/tournament_1/checkpoints_mcts100_cpuct2'
weight_fn = 'checkpoint_39.pth.tar'
example_fn = 'checkpoint_41.pth.tar'
load_folder_file = (weight_dir,example_fn)

modelFile = os.path.join(load_folder_file[0], load_folder_file[1])
examplesFile = modelFile + ".examples"
with open(examplesFile, "rb") as f:
    trainExamplesHistory = Unpickler(f).load()


from random import shuffle
trainExamples = []
for e in trainExamplesHistory:
    trainExamples.extend(e)
shuffle(trainExamples)



args = bnp.args
args.batch_size = 64



tE = trainExamples[:N_trainexs//args.batch_size * args.batch_size]
N_trainexs = len(tE)

# input_boards, target_pis, target_vs = list(zip(*trainExamples[:10]))
# input_boards = np.asarray(input_boards)
# target_pis = np.asarray(target_pis)
# target_vs = np.asarray(target_vs)

In [59]:
# create agent before training:

# args = bnp.args
# args.batch_size = 64
assert N_trainexs % args.batch_size ==0

pnet = NNetWrapper(game)
pnet.load_checkpoint(folder=weight_dir, filename=weight_fn)

nnet = NNetWrapper(game,args=args)
nnet.load_checkpoint(folder=weight_dir, filename=weight_fn)

reload(bnp)
nnet_pvcorr = bnp.NNetWrapper_pvcorr(game,args=args)
nnet_pvcorr.load_checkpoint(folder=weight_dir, filename=weight_fn)

nnet_dict = {'p':pnet,'n':nnet,'n_pvcorr':nnet_pvcorr}

In [60]:
# training
for pl, val in nnet_dict.items():
    if pl!='p':
        print(f'player {pl}')
        val.train(tE)

player n
Epoch 1/10
2249/2249 [==============================] - 17s 8ms/step - loss: 1.0545 - pi_loss: 0.9125 - v_loss: 0.1421
Epoch 2/10
2249/2249 [==============================] - 17s 8ms/step - loss: 0.9990 - pi_loss: 0.8622 - v_loss: 0.1368
Epoch 3/10
2249/2249 [==============================] - 17s 8ms/step - loss: 0.9864 - pi_loss: 0.8523 - v_loss: 0.1342
Epoch 4/10
2249/2249 [==============================] - 17s 8ms/step - loss: 0.9812 - pi_loss: 0.8490 - v_loss: 0.1322
Epoch 5/10
2249/2249 [==============================] - 17s 8ms/step - loss: 0.9771 - pi_loss: 0.8464 - v_loss: 0.1307
Epoch 6/10
2249/2249 [==============================] - 17s 8ms/step - loss: 0.9725 - pi_loss: 0.8426 - v_loss: 0.1299
Epoch 7/10
2249/2249 [==============================] - 17s 8ms/step - loss: 0.9697 - pi_loss: 0.8409 - v_loss: 0.1289
Epoch 8/10
2249/2249 [==============================] - 17s 8ms/step - loss: 0.9664 - pi_loss: 0.8378 - v_loss: 0.1286
Epoch 9/10
2249/2249 [=================

In [61]:
# create mcts and agents 
n_mcts = 100
cpuct = 2
tree_args = dotdict({
'numMCTSSims': n_mcts,
'cpuct': cpuct,
})
tree_dict = {}
ai_dict = {}
for player,val_func in nnet_dict.items():
    tree_dict[player] = MCTS(game,val_func,tree_args)
    ai_dict[player] = lambda x: np.random.choice(np.arange(game.getActionSize()),p=tree_dict[player].getActionProb(x, temp=1/10))



In [62]:
participants = [['p','n'],['p','n_pvcorr'],['n','n_pvcorr']]
res_l = []
ngames = 10
for p1,p2 in participants:
    print(f'{p1} vs {p2}')
    arena = Arena(ai_dict[p1],ai_dict[p2],game,display=game.display,tree1=tree_dict[p1],tree2=tree_dict[p2])
    res=arena.playGames(ngames,verbose=True)
    res_l.append(res)
    print(f'{p1} vs {p2}')
    print(f'result {res}')
    
for (p1,p2),res in zip(participants,res_l):
    print(f'{p1} vs {p2}')
    print(f'result {res}')

Arena.playGames (1):   0%|          | 0/5 [00:00<?, ?it/s]

p vs n
Turn  1 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - - - - - - - |
2 |- - - - - - - - - |
3 |- - - - - - - - - |
-----------------------
Turn  2 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - O - - - - - |
1 |- - - - - - - - - |
2 |- - - - - - - - - |
3 |- - - - - - - - - |
-----------------------
Turn  3 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - O - - - - - |
1 |- - - - X - - - - |
2 |- - - - - - - - - |
3 |- - - - - - - - - |
-----------------------
Turn  4 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - O - - - - - |
1 |- - O - X - - - - |
2 |- - - - - - - - - |
3 |- - - - - - - - - |
-----------------------
Turn  5 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - O - - - - - |
1 |- - O - X - - - - |
2 |- X - - - - - - - |
3 |- - - - - - - - - |
-----------------------
Turn  6 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - O - - - - - |
1 |

Arena.playGames (1):  20%|██        | 1/5 [00:25<01:40, 25.17s/it]

Turn  21 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- X X O O O - X - |
1 |- - O X X X O - - |
2 |- X - O X O - - - |
3 |- - X O O O X - - |
-----------------------
Game over: Turn  21 Result  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- X X O O O O X - |
1 |- - O X X X O - - |
2 |- X - O X O - - - |
3 |- - X O O O X - - |
-----------------------
Turn  1 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - - - - - - - |
2 |- - - - - - - - - |
3 |- - - - - - - - - |
-----------------------
Turn  2 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - - - - - - - |
2 |- - - - - - - - - |
3 |- - - - - O - - - |
-----------------------
Turn  3 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - - - - - - - |
2 |- - - - X - - - - |
3 |- - - - - O - - - |
-----------------------
Turn  4 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - 

Arena.playGames (1):  40%|████      | 2/5 [00:42<01:08, 22.92s/it]

...

Turn  23 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |X X - O - O - X - |
1 |- O O X X X O - - |
2 |- X O O X O - - - |
3 |- X X O O O X - - |
-----------------------
Turn  24 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |X X - O O O - X - |
1 |- O O X X X O - - |
2 |- X O O X O - - - |
3 |- X X O O O X - - |
-----------------------
Turn  25 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |X X X O O O - X - |
1 |- O O X X X O - - |
2 |- X O O X O - - - |
3 |- X X O O O X - - |
-----------------------
Game over: Turn  25 Result  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |X X X O O O O X - |
1 |- O O X X X O - - |
2 |- X O O X O - - - |
3 |- X X O O O X - - |
-----------------------
Turn  1 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - - - - - - - |
2 |- - - - - - - - - |
3 |- - - - - - - - - |
-----------------------
Turn  2 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - O - - - - 

Arena.playGames (2):  80%|████████  | 4/5 [01:43<00:25, 25.94s/it]

Turn  21 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- X X O O O - X - |
1 |- - O X X X O - - |
2 |- X - O X O - - - |
3 |- - X O O O X - - |
-----------------------
Game over: Turn  21 Result  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- X X O O O O X - |
1 |- - O X X X O - - |
2 |- X - O X O - - - |
3 |- - X O O O X - - |
-----------------------
Turn  1 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - - - - - - - |
2 |- - - - - - - - - |
3 |- - - - - - - - - |
-----------------------
Turn  2 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - - - - - - - |
2 |- - - - - - - - - |
3 |- - - O - - - - - |
-----------------------
Turn  3 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - - - - - - - |
2 |- - - - X - - - - |
3 |- - - O - - - - - |
-----------------------
Turn  4 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - 

Arena.playGames (2): 100%|██████████| 5/5 [02:15<00:00, 27.04s/it]

Turn  35 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |X O X O O X O X - |
1 |X X O O O X O O O |
2 |X X O X X O X O X |
3 |O X X O O X X O - |
-----------------------
Turn  36 Player  -1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |X O X O O X O X - |
1 |X X O O O X O O O |
2 |X X O X X O X O X |
3 |O X X O O X X O O |
-----------------------
Game over: Turn  36 Result  0.0001
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |X O X O O X O X X |
1 |X X O O O X O O O |
2 |X X O X X O X O X |
3 |O X X O O X X O O |
-----------------------
n vs n_pvcorr
result (1, 2, 7)
p vs n
result (2, 1, 7)
p vs n_pvcorr
result (2, 1, 7)
n vs n_pvcorr
result (1, 2, 7)


In [63]:
for (p1,p2),res in zip(participants,res_l):
    print(f'{p1} vs {p2}')
    print(f'result {res}')

p vs n
result (2, 1, 7)
p vs n_pvcorr
result (2, 1, 7)
n vs n_pvcorr
result (1, 2, 7)


In [10]:
for (p1,p2),res in zip(participants,res_l):
    print(f'{p1} vs {p2}')
    print(f'result {res}')

p vs n
result (3, 5, 2)
p vs n_pvcorr
result (3, 6, 1)
n vs n_pvcorr
result (2, 3, 5)


In [51]:
for pl,val in nnet_dict.items():
    tosavefolder = '/scratch/zz737/fiar/tournaments/temp_test'
    filename=f'{pl}.pth.tar'
    val.save_checkpoint(folder=tosavefolder,filename=filename)

Checkpoint Directory exists! 
Checkpoint Directory exists! 
Checkpoint Directory exists! 
